In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import corpus
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from nltk import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error



In [ ]:
Data = pd.read_csv("D:\Machine Learning\dataset\Amazontrain.csv")

In [ ]:
Data.head(10)

In [ ]:
Data.info()

In [ ]:
Data.isnull().sum()

In [ ]:
Data_new = Data.drop(columns=['PRODUCT_ID',"BULLET_POINTS",'DESCRIPTION'], axis= 1)

In [ ]:
Data_new.isnull().sum()

In [ ]:
Data_new = Data_new.dropna()
Data_new

In [ ]:
Data_new.duplicated().sum()
Data_new= Data_new.drop_duplicates(keep='first')
Data_new

In [ ]:
Data_new['PRODUCT_TYPE_ID'].value_counts().head(500)

In [ ]:
count = Data_new['PRODUCT_TYPE_ID'].value_counts()
threshold = 1000
repl = count[count <= threshold].index
print(Data_new['PRODUCT_TYPE_ID'].unique())

In [ ]:
Data_cat = pd.get_dummies(Data_new['PRODUCT_TYPE_ID'].replace(repl,"Uncommmon"))


In [ ]:
nltk.download('stopwords')


In [ ]:
Data_cat

In [ ]:
import string 

In [ ]:
ps = PorterStemmer()


In [ ]:
def preprocessing(contents):
    stemmed_content = re.sub('[^a-zA-Z]',' ',(contents))
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content
    
    
    

In [ ]:
Data_new_sample = Data_new.sample(20000, random_state=2)

In [ ]:
Data_new_sample

In [ ]:
Data_new_sample['TITLE'] = Data_new_sample['TITLE'].apply(preprocessing)

In [ ]:
Data_new_sample

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)


In [ ]:
x  = Data_new_sample.loc[:,['TITLE','PRODUCT_TYPE_ID']]

In [ ]:
y = Data_new_sample['PRODUCT_LENGTH']

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)

In [ ]:
titles = list(x['TITLE'])

title_arr = (vectorizer.fit_transform(titles)).toarray()
title_arr


In [ ]:
temp_df = pd.DataFrame(title_arr, index=x.index)


In [ ]:
temp_df

In [ ]:
x = x.drop('TITLE', axis =1)

In [ ]:
final_data = pd.concat([x,temp_df],axis =1)


In [ ]:
final_data

In [ ]:
final_data.columns = final_data.columns.astype(str)

In [ ]:
x_train, x_test, y_train,y_test = train_test_split(final_data, y, test_size = 0.25, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
score = r2_score(y_test, y_predicted)
score


In [ ]:
mse = mean_squared_error(y_test, y_predicted)
mse

In [ ]:
mae = mean_absolute_error(y_test, y_predicted)
mae

In [ ]:
maep = mean_absolute_percentage_error(y_test, y_predicted)
maep